In [1]:
# -------------------------------------------------------------
# TRANSACTION-BASED PREPROCESSING & FEATURE ENGINEERING
# -------------------------------------------------------------

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib

In [2]:
# -------------------------------------------------------------
# 1. LOAD DATA
# -------------------------------------------------------------

df = pd.read_csv("C:\\Users\\user\\Desktop\\LinkLock_anomalydetection\\LinkLock\\Link-Lock-Fraud-Anomaly-Detection-system-\\backend\\app\ml\\bank_transactions_data_1.csv")
print("Dataset Loaded:")
print(df.head())

Dataset Loaded:
  TransactionID AccountID  TransactionAmount   TransactionDate  \
0      TX000001   AC00128              14.09   4/11/2023 16:29   
1      TX000002   AC00455             376.24   6/27/2023 16:44   
2      TX000003   AC00019             126.29   7/10/2023 18:16   
3      TX000004   AC00070             184.50    5/5/2023 16:32   
4      TX000005   AC00411              13.45  10/16/2023 17:51   

  TransactionType   Location DeviceID      IP Address MerchantID Channel  \
0           Debit  San Diego  D000380  162.198.218.92       M015     ATM   
1           Debit    Houston  D000051     13.149.61.4       M052     ATM   
2           Debit       Mesa  D000235  215.97.143.157       M009  Online   
3           Debit    Raleigh  D000187  200.13.225.150       M002  Online   
4          Credit    Atlanta  D000308    65.164.3.100       M091  Online   

   CustomerAge CustomerOccupation  TransactionDuration  LoginAttempts  \
0           70             Doctor                   81   

In [3]:
# -------------------------------------------------------------
# 2. CONVERT DATE COLUMNS
# -------------------------------------------------------------

df["TransactionDate"] = pd.to_datetime(df["TransactionDate"], errors="coerce")
df["PreviousTransactionDate"] = pd.to_datetime(df["PreviousTransactionDate"], errors="coerce")

# Sort by date per account
df = df.sort_values(by=["AccountID", "TransactionDate"])

In [4]:
# -------------------------------------------------------------
# 3. TIME-BASED FEATURES
# -------------------------------------------------------------

df["hour"] = df["TransactionDate"].dt.hour
df["day_of_week"] = df["TransactionDate"].dt.dayofweek
df["day"] = df["TransactionDate"].dt.day

# Time between this and last transaction
df["time_since_last"] = df.groupby("AccountID")["TransactionDate"].diff().dt.total_seconds()

# Replace missing with median
df["time_since_last"] = df["time_since_last"].fillna(df["time_since_last"].median())


In [5]:
# -------------------------------------------------------------
# 4. TRANSACTION FREQUENCY FEATURES
# -------------------------------------------------------------

# Ensure data is sorted for merge_asof
df = df.sort_values(["AccountID", "TransactionDate"]).reset_index(drop=True)

# Initialize new columns
df["tx_last_1hr"] = 0
df["tx_last_24hr"] = 0

# Loop through each account independently
for acc in df["AccountID"].unique():

    # Subset data for one user
    sub = df[df["AccountID"] == acc].copy()

    # Create a helper frame for counting
    helper = sub[["TransactionDate"]].copy()
    helper["count"] = 1  # each row = 1 transaction

    # --- Calculate transactions in last 1 hour ---
    merged_1h = pd.merge_asof(
        sub.sort_values("TransactionDate"),
        helper.sort_values("TransactionDate"),
        on="TransactionDate",
        direction="backward",
        tolerance=pd.Timedelta("1h")
    )

    df.loc[sub.index, "tx_last_1hr"] = merged_1h["count"].cumsum()

    # --- Calculate transactions in last 24 hours ---
    merged_24h = pd.merge_asof(
        sub.sort_values("TransactionDate"),
        helper.sort_values("TransactionDate"),
        on="TransactionDate",
        direction="backward",
        tolerance=pd.Timedelta("24h")
    )

    df.loc[sub.index, "tx_last_24hr"] = merged_24h["count"].cumsum()


In [6]:

# -------------------------------------------------------------
# 5. AMOUNT-BASED FEATURES
# -------------------------------------------------------------

# Z-score per user
df["amount_zscore"] = df.groupby("AccountID")["TransactionAmount"].transform(
    lambda x: (x - x.mean()) / x.std()
)

df["amount_zscore"] = df["amount_zscore"].fillna(0)

# Ratio to account balance
df["amount_to_balance"] = df["TransactionAmount"] / (df["AccountBalance"] + 1)


In [8]:

# -------------------------------------------------------------
# 6. DEVICE/IP/LOCATION FEATURES
# -------------------------------------------------------------

# Device change
df["prev_device"] = df.groupby("AccountID")["DeviceID"].shift(1)
df["device_changed"] = (df["DeviceID"] != df["prev_device"]).astype(int)

# IP change
df["prev_ip"] = df.groupby("AccountID")["IP Address"].shift(1)
df["ip_changed"] = (df["IP Address"] != df["prev_ip"]).astype(int)

# Location change
df["prev_loc"] = df.groupby("AccountID")["Location"].shift(1)
df["location_changed"] = (df["Location"] != df["prev_loc"]).astype(int)

# Fill missing
df[["device_changed", "ip_changed", "location_changed"]] = df[["device_changed", "ip_changed", "location_changed"]].fillna(0)

In [9]:

# -------------------------------------------------------------
# 7. CATEGORY ENCODING
# -------------------------------------------------------------

categorical_cols = [
    "TransactionType",
    "Location",
    "DeviceID",
    "IP Address",
    "MerchantID",
    "Channel",
    "CustomerOccupation"
]

encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

joblib.dump(encoders, "transaction_encoders.pkl")


['transaction_encoders.pkl']

In [10]:
# -------------------------------------------------------------
# 8. SELECT FINAL TRANSACTION FEATURES
# -------------------------------------------------------------

feature_cols = [
    "TransactionAmount",
    "hour", "day_of_week", "day",
    "time_since_last",
    "tx_last_1hr", "tx_last_24hr",
    "amount_zscore",
    "amount_to_balance",
    "device_changed", "ip_changed", "location_changed",
    "TransactionType", "Location", "DeviceID",
    "IP Address", "MerchantID", "Channel",
    "CustomerAge", "LoginAttempts", "AccountBalance"
]

X = df[feature_cols]

In [11]:
# -------------------------------------------------------------
# 9. SCALING
# -------------------------------------------------------------

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

joblib.dump(scaler, "transaction_scaler.pkl")

['transaction_scaler.pkl']

In [12]:
# -------------------------------------------------------------
# 10. SAVE FINAL PREPROCESSED DATA
# -------------------------------------------------------------

final_df = pd.DataFrame(X_scaled, columns=feature_cols)
final_df.to_csv("transaction_features.csv", index=False)

print("\nTransaction-based preprocessing complete!")
print("Saved: transaction_features.csv, transaction_scaler.pkl, transaction_encoders.pkl")


Transaction-based preprocessing complete!
Saved: transaction_features.csv, transaction_scaler.pkl, transaction_encoders.pkl
